In [17]:
from pydantic import Field

from dslmodel import DSLModel, init_instant
from typing import List, Type
from functools import reduce
import logging

def from_prompt_chain(initial_prompt: str, models: List[Type[DSLModel]]) -> List[DSLModel]:
    """
    Executes a chain of DSLModel.from_prompt calls, where the result of one is passed as the prompt to the next.

    Args:
        initial_prompt (str): The initial prompt to start the chain.
        models (List[Type[DSLModel]]): A list of DSLModel subclasses to be instantiated in sequence.

    Returns:
        List[DSLModel]: A list of instantiated DSLModel objects resulting from the chain.
    """
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO)

    def chain(accumulator: List[DSLModel], model_class: Type[DSLModel]) -> List[DSLModel]:
        """
        Helper function for reduce to execute from_prompt and accumulate results.

        Args:
            accumulator (List[DSLModel]): The list of previously instantiated models.
            model_class (Type[DSLModel]): The current DSLModel subclass to instantiate.

        Returns:
            List[DSLModel]: Updated list of instantiated models.
        """
        # Determine the prompt for the current model
        if not accumulator:
            prompt = initial_prompt
            logger.debug(f"Initial prompt for {model_class.__name__}: {prompt}")
        else:
            # Assuming each model has a 'generated_text' attribute or similar
            # Adjust the attribute name based on your DSLModel definitions
            previous_model = accumulator[-1]
            prompt = str(previous_model)
            logger.debug(f"Prompt for {model_class.__name__} from previous model: {prompt}")

        try:
            # Instantiate the current model using from_prompt
            model_instance = model_class.from_prompt(prompt)
            logger.info(f"Successfully instantiated {model_class.__name__}")
            return accumulator + [model_instance]
        except Exception as e:
            logger.error(f"Error instantiating {model_class.__name__} with prompt '{prompt}': {e}")
            # Depending on requirements, you can choose to append None or skip
            return accumulator + [None]

    # Use reduce to apply the chain function across the models list
    results = reduce(chain, models, [])

    return results


In [18]:
from typing import Optional

class Task(DSLModel):
    """
    Represents a task within a work session.
    """
    name: str = Field(..., description="The name of the task.")
    description: Optional[str] = Field(None, description="A brief description of the task.")
    duration_minutes: int = Field(..., description="Estimated duration to complete the task.")
    status: str = Field("Pending", description="Current status of the task (e.g., Pending, In Progress, Completed).")


class SMARTGoal(DSLModel):
    """
    Represents a SMART (Specific, Measurable, Achievable, Relevant, Time-bound) goal.
    """
    title: str = Field(..., description="The title of the SMART goal.")
    specific: str = Field(..., description="Specific details of the goal.")
    measurable: str = Field(..., description="Metrics to measure the goal's progress.")
    achievable: str = Field(..., description="Criteria that make the goal achievable.")
    relevant: str = Field(..., description="Relevance of the goal to broader objectives.")
    time_bound: str = Field(..., description="Time frame to achieve the goal.")


In [19]:
# Define the initial prompt
initial_prompt = "Grocery shopping for milk, cheese, and bread."

# Define the list of models in the desired sequence
models_chain = [SMARTGoal, Task]

init_instant()

# Execute the from_prompt_chain function
chain_results = from_prompt_chain(initial_prompt, models_chain)

# Access and print the results
for idx, result in enumerate(chain_results):
    if result:
        print(f"Result {idx + 1} ({result.__class__.__name__}): {result.generated_text}")
    else:
        print(f"Result {idx + 1}: Failed to instantiate the model.")
